# Tidying up our data - Part 2
## Flattening a nested schema

As usual, we'll start by loading the data.  
Because we expect you to know how to setup things so that you can load data from S3, we're doing it for you now.  
Make sure you go through our code and check that you were actually using our best practices.

In [0]:
items_exploded_path = "s3://full-stack-bigdata-datasets/Big_Data/YOUTUBE/items_exploded.json"
df = spark.read.json(items_exploded_path)

1. As a sanity check, count the rows in the DataFrame

In [0]:
df.count()

Out[3]: 3907

2. Print out the schema of the DataFrame

In [0]:
df.printSchema()

root
 |-- items: struct (nullable = true)
 |    |-- contentDetails: struct (nullable = true)
 |    |    |-- caption: string (nullable = true)
 |    |    |-- contentRating: struct (nullable = true)
 |    |    |    |-- ytRating: string (nullable = true)
 |    |    |-- definition: string (nullable = true)
 |    |    |-- dimension: string (nullable = true)
 |    |    |-- duration: string (nullable = true)
 |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |-- projection: string (nullable = true)
 |    |    |-- regionRestriction: struct (nullable = true)
 |    |    |    |-- allowed: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- blocked: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |-- etag: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- kind: string (nullable = true)
 |    |-- snippet: struct (nullable = true)
 |    |    |-- categoryId: st

In [0]:
df.columns

Out[8]: ['items']

### Working with the schema

We're ready to get started :)

Our schema is like a tree, we want to collect all its leaves and put them neatly as columns of our DataFrame.  
That's called **flattening a schema**.

Let's give it a try with the `title` element inside the `items` columns, then into the nested field `snippet`, and finally in the nested subfield `title`.

You can do this using the `.getField()` column method [documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.getField).

3. Select the `title` subfield from the `snippet` subfield in the `items` column show the first 5 elements

In [0]:
from pyspark.sql import functions as F
df.withColumn('items', F.col('items').getField('snippet').getField('title')).show()

+--------------------+
|               items|
+--------------------+
|VOLO. "L'air d'un...|
|Julian Jeweil - A...|
| Justice - D.A.N.C.E|
|Gramatik - Tortur...|
|Ben Howard - Oats...|
|RA Sessions: DARK...|
| Dusty Kid - America|
|MØ - Waste Of Tim...|
|[Electronic] - Pe...|
|Always Coca Cola ...|
|Dom La Nena  03 V...|
|Lazare Hoche - Lu...|
|Liars - The Other...|
|NTM - That's my p...|
|La maman et la pu...|
|The Strokes - Aut...|
|Skyden feat. Sask...|
|Gift of Gab - El ...|
|Down in Mexico by...|
|Phoenix - If I Ev...|
+--------------------+
only showing top 20 rows



In [0]:
df.select(F.col('items').getField('snippet').getField('title')).show()

+--------------------+
| items.snippet.title|
+--------------------+
|VOLO. "L'air d'un...|
|Julian Jeweil - A...|
| Justice - D.A.N.C.E|
|Gramatik - Tortur...|
|Ben Howard - Oats...|
|RA Sessions: DARK...|
| Dusty Kid - America|
|MØ - Waste Of Tim...|
|[Electronic] - Pe...|
|Always Coca Cola ...|
|Dom La Nena  03 V...|
|Lazare Hoche - Lu...|
|Liars - The Other...|
|NTM - That's my p...|
|La maman et la pu...|
|The Strokes - Aut...|
|Skyden feat. Sask...|
|Gift of Gab - El ...|
|Down in Mexico by...|
|Phoenix - If I Ev...|
+--------------------+
only showing top 20 rows



That's it, easy peasy 🙂.

We could just keep doing this for every single leaf of the schema and we're done.

I don't know about you, but I think this is incredebly **boring**. Also, what if tomorrow, Youtube adds a new leaf to its API results?

Come on, we're programmers, we're **supposed to automate stuff, aren't we?**

What we need is a way to build that list of leaves... 
Not gonna lie, it's not trivial, it's called a tree traversal and this is beyond the scope of this course.

Which means we will do this part for you. In the following cell, we've included a function called `walkSchema`. What this functions does, is that it walk the schema of our DataFrame with a nested schema and harvest its leave. Returning them with full path like this `items.snippet.title` as a string.

Well, "returning", not exactly. But we will see about that later.

**[TODO]**
Take a look at the function, you're not supposed to understand what it does, this is beyond the scope of this course.  
But when you're learning, it's always a good idea to be exposed to new things.

In [0]:
# Let's give you the intuition for the flattening function we will share with you now
# The idea is to automatically dig deeper and deeper into the schema in order to extract
# all the column names in the form "array1.array2.array3.field1", let's go!

# we'll work with the schema in json format, which will be way easier to manipulate
df.schema.jsonValue()

# It's nothing more than a dictionnary with keys

Out[15]: {'type': 'struct',
 'fields': [{'name': 'items',
   'type': {'type': 'struct',
    'fields': [{'name': 'contentDetails',
      'type': {'type': 'struct',
       'fields': [{'name': 'caption',
         'type': 'string',
         'nullable': True,
         'metadata': {}},
        {'name': 'contentRating',
         'type': {'type': 'struct',
          'fields': [{'name': 'ytRating',
            'type': 'string',
            'nullable': True,
            'metadata': {}}]},
         'nullable': True,
         'metadata': {}},
        {'name': 'definition',
         'type': 'string',
         'nullable': True,
         'metadata': {}},
        {'name': 'dimension',
         'type': 'string',
         'nullable': True,
         'metadata': {}},
        {'name': 'duration',
         'type': 'string',
         'nullable': True,
         'metadata': {}},
        {'name': 'licensedContent',
         'type': 'boolean',
         'nullable': True,
         'metadata': {}},
        {'name':

In [0]:
df.schema.jsonValue().keys()
# Only two keys at this stage, type and fields, let's explore the type key

Out[16]: dict_keys(['type', 'fields'])

In [0]:
df.schema.jsonValue()["type"]
# The value associated is struct

Out[17]: 'struct'

In [0]:
# let's explore the content of the other key
df.schema.jsonValue()["fields"]

Out[18]: [{'name': 'items',
  'type': {'type': 'struct',
   'fields': [{'name': 'contentDetails',
     'type': {'type': 'struct',
      'fields': [{'name': 'caption',
        'type': 'string',
        'nullable': True,
        'metadata': {}},
       {'name': 'contentRating',
        'type': {'type': 'struct',
         'fields': [{'name': 'ytRating',
           'type': 'string',
           'nullable': True,
           'metadata': {}}]},
        'nullable': True,
        'metadata': {}},
       {'name': 'definition',
        'type': 'string',
        'nullable': True,
        'metadata': {}},
       {'name': 'dimension',
        'type': 'string',
        'nullable': True,
        'metadata': {}},
       {'name': 'duration',
        'type': 'string',
        'nullable': True,
        'metadata': {}},
       {'name': 'licensedContent',
        'type': 'boolean',
        'nullable': True,
        'metadata': {}},
       {'name': 'projection',
        'type': 'string',
        'nullable': T

In [0]:
#it's a list, what's the first element?
df.schema.jsonValue()["fields"][0]

Out[19]: {'name': 'items',
 'type': {'type': 'struct',
  'fields': [{'name': 'contentDetails',
    'type': {'type': 'struct',
     'fields': [{'name': 'caption',
       'type': 'string',
       'nullable': True,
       'metadata': {}},
      {'name': 'contentRating',
       'type': {'type': 'struct',
        'fields': [{'name': 'ytRating',
          'type': 'string',
          'nullable': True,
          'metadata': {}}]},
       'nullable': True,
       'metadata': {}},
      {'name': 'definition',
       'type': 'string',
       'nullable': True,
       'metadata': {}},
      {'name': 'dimension',
       'type': 'string',
       'nullable': True,
       'metadata': {}},
      {'name': 'duration', 'type': 'string', 'nullable': True, 'metadata': {}},
      {'name': 'licensedContent',
       'type': 'boolean',
       'nullable': True,
       'metadata': {}},
      {'name': 'projection',
       'type': 'string',
       'nullable': True,
       'metadata': {}},
      {'name': 'regionRestr

In [0]:
# what keys does it have ?
df.schema.jsonValue()["fields"][0].keys()

Out[20]: dict_keys(['name', 'type', 'nullable', 'metadata'])

In [0]:
# the key name contains the name of the field
df.schema.jsonValue()["fields"][0]["name"]

Out[21]: 'items'

In [0]:
# if we have the key type then we have subfields inside it
df.schema.jsonValue()["fields"][0]["type"]
# and we are back to the same structure we had at the beginning and we can start digging again
# that's the spirit of the function below

Out[22]: {'type': 'struct',
 'fields': [{'name': 'contentDetails',
   'type': {'type': 'struct',
    'fields': [{'name': 'caption',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'contentRating',
      'type': {'type': 'struct',
       'fields': [{'name': 'ytRating',
         'type': 'string',
         'nullable': True,
         'metadata': {}}]},
      'nullable': True,
      'metadata': {}},
     {'name': 'definition',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'dimension', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'duration', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': 'licensedContent',
      'type': 'boolean',
      'nullable': True,
      'metadata': {}},
     {'name': 'projection',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': 'regionRestriction',
      'type': {'type': 'struct',
       'fields': [{'name': 'allowed

In [0]:
from pyspark.sql.types import StructType, StructField
from typing import List, Dict, Generator, Union, Callable

# This is actually written like a scala function, we'll walk you through it
def walkSchema(schema: Union[StructType, StructField]) -> Generator[str, None, None]:
    """Explores a PySpark schema:
    
    schema: StructType | StructField
    
    Yield
    -----
    A generator of strings, the name of each field in the schema
    """
    
    # we define a function _walk that produces a string generator from
    # a dictionnary "schema_dct", and a string "prefix"
    def _walk(schema_dct: Dict['str', Union['str', list, dict]],
              prefix: str = "") -> Generator[str, None, None]:
        assert isinstance(prefix, str), "prefix should be a string" # check if prefix is a string
        
        # this function returns "name" if there's no prefix and "prefix.name" if prefix exists
        fullName: Callable[str, str] = lambda name: ( 
            name if not prefix else f"{prefix}.{name}")
        
        # we get the next name one level lower from the dictionnary
        name = schema_dct.get('name', '')
        
        # if the type is struct then we search for the fields key
        # if fields is there we apply the function again and dig one level deeper in
        # the schema and set a prefix
        if schema_dct['type'] == 'struct':
            assert 'fields' in schema_dct, (
                "It's a StructType, we should have some fields")
            for field in schema_dct['fields']:
                yield from _walk(field, prefix=prefix)
        # if we have a dict type and we can't find fields then we
        # dig one level deeper and apply the _walk function again
        elif isinstance(schema_dct['type'], dict):
            assert 'fields' not in schema_dct, (
                "We're missing some keys here")
            yield from _walk(schema_dct['type'], prefix=fullName(name))
        # If we finally reached the end and found a name we yield the full name
        elif name:
            yield fullName(name)
    
    yield from _walk(schema.jsonValue())

# yield as opposed to return, returns a result but does not stop the function from running, it keeps
# running even after returning one result.

We will give this function a try, and see how it behaves...  
You might have to look into PySpark documentation to learn how to access the schema of a DataFrame.

5. Call `walkSchema(...)` on our dataframe schema: `col_names` then print it out to the screen

In [0]:
walkSchema(df.schema)

Out[28]: <generator object walkSchema at 0x7fb8c6c52c10>

You should see an output similar to `<generator object walkSchema at 0x7f9eb0e390c0>`.  
It's a Python's generator, you can read more about it [here](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/).

For now, you just have to know, that just like a python's `list`, a `generator` is also `iterable`, which means we can iterate over it with a `for` loop.

```python
for e in my_generator:
    # You can access each element of the generator here
```

We'll give it a try, by printing out the values of our col_names.

6. Iterate over the walked schema

*NOTE: give the name `col_name` to the iterating variable*

In [0]:
for e in walkSchema(df.schema):
    print(e)

items.contentDetails.caption
items.contentDetails.contentRating.ytRating
items.contentDetails.definition
items.contentDetails.dimension
items.contentDetails.duration
items.contentDetails.licensedContent
items.contentDetails.projection
items.etag
items.id
items.kind
items.snippet.categoryId
items.snippet.channelId
items.snippet.channelTitle
items.snippet.defaultAudioLanguage
items.snippet.defaultLanguage
items.snippet.description
items.snippet.liveBroadcastContent
items.snippet.localized.description
items.snippet.localized.title
items.snippet.publishedAt
items.snippet.thumbnails.default.height
items.snippet.thumbnails.default.url
items.snippet.thumbnails.default.width
items.snippet.thumbnails.high.height
items.snippet.thumbnails.high.url
items.snippet.thumbnails.high.width
items.snippet.thumbnails.maxres.height
items.snippet.thumbnails.maxres.url
items.snippet.thumbnails.maxres.width
items.snippet.thumbnails.medium.height
items.snippet.thumbnails.medium.url
items.snippet.thumbnails.medi

Perfect, that's all the leafs of our schema.  
And we can just repeat the work we did with `items.snippet.title` for every column of this list.


There are a couple ways to do this, you've got at least 2 options (using standard "non-functionnal" python):
- build a list comprehension (or unpack the generator) and pass it to a `.select(...)` statement
- iterate over the generator, and use `.withColumn(...)`

_But our favorite uses a functional approach. It particularly makes sense because Spark is based on Scala, a functionnal language.  
If you're interested in this approach, take a look at `reduce` from the `functools` package in Python.  
In this simple isolated case, it actually makes things look a bit harder than they should, but it would make it easier to neatly integrate this step in a global pipeline.  
**Beware, if you're not familiar with functional programming that will probably feel non-trivial.**_

In [0]:
from functools import reduce
from pyspark.sql import functions as F

# Non-functional way: unpacking the generator
# exploded_df = df.select(*walkSchema(df.schema))

# The functional way, using functools' reduce
exploded_df = reduce(lambda memo_df, col_name : memo_df.withColumn(col_name, F.col(col_name)), walkSchema(df.schema), df).drop('items')

exploded_df.limit(5).toPandas()

,items.contentDetails.caption,items.contentDetails.contentRating.ytRating,items.contentDetails.definition,items.contentDetails.dimension,items.contentDetails.duration,items.contentDetails.licensedContent,items.contentDetails.projection,items.etag,items.id,items.kind,...,items.statistics.dislikeCount,items.statistics.favoriteCount,items.statistics.likeCount,items.statistics.viewCount,items.status.embeddable,items.status.license,items.status.madeForKids,items.status.privacyStatus,items.status.publicStatsViewable,items.status.uploadStatus
0,false,None,sd,2d,PT3M33S,True,rectangular,SqP7uUVSol30dxvuScN6JUny6T4,t1l8Z6gLPzo,youtube#video,...,26,0,1028,223172,True,youtube,False,public,True,processed
1,false,None,hd,2d,PT7M46S,False,rectangular,m3DnhzTEw9ABiqzBvdasfk5Av_8,we5gzZq5Avg,youtube#video,...,3,0,124,13409,True,youtube,False,public,True,processed
2,false,None,sd,2d,PT3M7S,False,rectangular,zyzs7STAR3NG-_pZe-0nGkbKoqg,49esza4eiK4,youtube#video,...,780,0,25540,10106655,True,youtube,False,public,True,processed
3,false,None,hd,2d,PT3M43S,False,rectangular,hX2C15F6fdO5A-stUFMU5Az2PvI,BoO6LfR7ca0,youtube#video,...,0,0,255,29153,True,youtube,False,public,True,processed
4,false,None,hd,2d,PT5M,False,rectangular,rYHoV38PLpMbRuX_zhGTVBKNotw,DaH4W1rY9us,youtube#video,...,1784,0,136033,16488714,True,youtube,False,public,True,processed


In [0]:
from pyspark.sql import functions as F

# Non-functional way: unpacking the generator
exploded_df = df.select(*walkSchema(df.schema))
exploded_df.limit(5).toPandas()

,caption,ytRating,definition,dimension,duration,licensedContent,projection,etag,id,kind,...,dislikeCount,favoriteCount,likeCount,viewCount,embeddable,license,madeForKids,privacyStatus,publicStatsViewable,uploadStatus
0,false,None,sd,2d,PT3M33S,True,rectangular,SqP7uUVSol30dxvuScN6JUny6T4,t1l8Z6gLPzo,youtube#video,...,26,0,1028,223172,True,youtube,False,public,True,processed
1,false,None,hd,2d,PT7M46S,False,rectangular,m3DnhzTEw9ABiqzBvdasfk5Av_8,we5gzZq5Avg,youtube#video,...,3,0,124,13409,True,youtube,False,public,True,processed
2,false,None,sd,2d,PT3M7S,False,rectangular,zyzs7STAR3NG-_pZe-0nGkbKoqg,49esza4eiK4,youtube#video,...,780,0,25540,10106655,True,youtube,False,public,True,processed
3,false,None,hd,2d,PT3M43S,False,rectangular,hX2C15F6fdO5A-stUFMU5Az2PvI,BoO6LfR7ca0,youtube#video,...,0,0,255,29153,True,youtube,False,public,True,processed
4,false,None,hd,2d,PT5M,False,rectangular,rYHoV38PLpMbRuX_zhGTVBKNotw,DaH4W1rY9us,youtube#video,...,1784,0,136033,16488714,True,youtube,False,public,True,processed


How amazing what we can do with a couple lines of well written code, isn't it?

Now that we're here, would be a good time to start analyzing the data we got. We will do this in the next assignment.

7. Save the output to S3 as a parquet file

In [0]:
out_path = "s3://full-stack-bigdata-datasets/Big_Data/40tude_tidy.parquet"
exploded_df.write.parquet(out_path, mode="overwrite")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-4290759733100622>:2
      1 out_path = "s3://full-stack-bigdata-datasets/Big_Data/40tude_tidy.parquet"
----> 2 exploded_df.write.parquet(out_path, mode="overwrite")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/lib/py